### Load preprocessed data

Run the script that downloads and processes the MovieLens data.
Uncomment it to run the download & processing script.

In [1]:
#!python ../src/download.py

In [2]:
import numpy as np
fh = np.load('data/dataset.npz', allow_pickle=True)

# We have a bunch of feature columns and last column is the y-target
# Note pytorch is finicky about need int64 types
train_x = fh['train_x'].astype(np.int64)
train_y = fh['train_y']
train_d = fh['train_dict']

# We've already split into train & test
test_x = fh['test_x'].astype(np.int64)
test_y = fh['test_y']
test_d = fh['test_dict']


n_user = int(fh['n_user'])
n_item = int(fh['n_item'])


In [4]:
from abstract_model import AbstractModel

In [9]:
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl

from pytorch_lightning.loggers import TensorBoardLogger


def l2_regularize(array):
    return torch.sum(array ** 2.0)


class MF(AbstractModel):
    def __init__(self, n_user, n_item, k=18, c_vector=1.0, c_bias=1.0, batch_size=128):
        super().__init__()
        # These are simple hyperparameters
        self.k = k
        self.n_user = n_user
        self.n_item = n_item
        self.c_vector = c_vector
        self.c_bias = c_bias
        self.batch_size = batch_size
        
        # These are learned and fit by PyTorch
        self.user = nn.Embedding(n_user, k)
        self.item = nn.Embedding(n_item, k)
        
        # We've added new terms here:
        self.bias_user = nn.Embedding(n_user, 1)
        self.bias_item = nn.Embedding(n_item, 1)
        self.bias = nn.Parameter(torch.ones(1))
    
    def forward(self, inputs):
        # This is the most import function in this script
        # These are the user indices, and correspond to "u" variable
        user_id = inputs[:, 0]
        # Item indices, correspond to the "i" variable
        item_id = inputs[:, 1]
        # vector user = p_u
        vector_user = self.user(user_id)
        # vector item = q_i
        vector_item = self.item(item_id)
        # this is a dot product & a user-item interaction: p_u * q_i
        ui_interaction = torch.sum(vector_user * vector_item, dim=1)
        
        # Pull out biases
        bias_user = self.bias_user(user_id).squeeze()
        bias_item = self.bias_item(item_id).squeeze()
        biases = (self.bias + bias_user + bias_item)

        # Add bias prediction to the interaction prediction
        prediction = ui_interaction + biases
        return prediction
    
    def likelihood(self, prediction, target):
        # MSE error between target = R_ui and prediction = p_u * q_i
        loss_mse = F.mse_loss(prediction, target.squeeze())
        return loss_mse
    
    def prior(self):
        # Add new regularization to the biases
        prior_bias_user =  l2_regularize(self.bias_user.weight) * self.c_bias
        prior_bias_item = l2_regularize(self.bias_item.weight) * self.c_bias
        # Compute L2 reularization over user (P) and item (Q) matrices
        prior_user =  l2_regularize(self.user.weight) * self.c_vector
        prior_item = l2_regularize(self.item.weight) * self.c_vector
        # Add up the MSE loss + user & item regularization
        total = prior_user + prior_item + prior_bias_user + prior_bias_item
        return total

GPU available: True, used: False
TPU available: False, using: 0 TPU cores


#### Optimize hyperparameters

In [16]:
import optuna
from pytorch_lightning.logging import WandbLogger

def objective(trial):
    # Sample parameters -- without declaring them in advance!
    k = trial.suggest_int('n_hid', 1, 20)
    c_vector = trial.suggest_loguniform('c_vector', 1e-8, 1e-1)
    c_bias = trial.suggest_loguniform('c_bias', 1e-8, 1e-1)
    model = MF(n_user, n_item, k=k, c_bias=c_bias, c_vector=c_vector)
    model.save_data(train_x, train_y, test_x, test_y)

    # add a logger
    logger = WandbLogger(name="02_mf", project="simple_mf")

    trainer = pl.Trainer(max_epochs=100, logger=logger,
                         early_stop_callback=True,
                         progress_bar_refresh_rate=10) 
    trainer.fit(model)
    results = trainer.test(model)
    return results['avg_test_loss']

In [ ]:
study = optuna.create_study()
study.optimize(objective, n_trials=10)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores


Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable

  | Name      | Type      | Params
----------------------------------------
0 | user      | Embedding | 120 K 
1 | item      | Embedding | 79 K  
2 | bias_user | Embedding | 6 K   
3 | bias_item | Embedding | 3 K   
